In [403]:
# -*- encoding: utf-8 -*-
import os
import sys

# 绘图包
import matplotlib
import matplotlib.pyplot as plt
# 导入相关包
%matplotlib inline

import numpy as np
import pandas as pd
import csv
import jieba
import re

In [404]:
# 读取CSV文件
def readDataFile(fileName,ptype):
    
    if ptype == 'train':
        # 训练数据4个字段
        COND = 4
    elif ptype == 'test':
        # 测试数据3个字段
        COND = 3

    file = open(filename)
    for line in file:   
        line = line.decode("utf8")
        l_ar = line.split("\t")
        
        if len(l_ar) == COND:
            id = l_ar[0]
            print "id is:",id
            lable = l_ar[3]
            print "lable is:",lable
            title = scan(l_ar[1])
            print "the title is:",title
            content = scan(l_ar[2])
            print "the content is :",content
                        
            # 标题分词                               
            seg_title = jieba.cut(title)
            print "seg_title:",seg_title
            seg_title_return = "\t".join(seg_title)
            
            # 正文分词
            seg_content = jieba.cut(content)
            print "seg_content:",seg_content
            seg_content_return = "\t".join(seg_content)

            if ptype == 'train':     
                # label = line[3]
                yield seg_title_return,seg_content_return,lable
            elif ptype == 'test':
                # doc_id = line[0]
                yield seg_title_return,seg_content_return,id

In [405]:
# 去除标点符号和特殊符号
def scan(line):
    string = re.sub("[+\s\:\-;?\"\.\!\-\/_,$%^*(+\"\']+|[+——！，“”·-▲▼▽•★ˇ<>「」。？、》《~@#￥%……&*（）]+".decode("utf8"), "".decode("utf8"),line)
    return string

In [406]:
def makeFile(num):
    
    dataPath = "/home/hadoop/Study/bdci_2017/BDCI2017-360/data/"

    # 训练数据
    train_outfile_title = dataPath + "train/train_output_title" + num + ".tsv"   
    train_outfile_content = dataPath + "train/train_output_content" + num + ".tsv"
    
    # 测试数据
#     test_outfile_title = dataPath + "test/test_output_title" + num + ".tsv"
#     test_outfile_content = dataPath + "test/test_output_content" + num + ".tsv"
    
    # 返回训练数据
    return(train_outfile_title,train_outfile_content)

    # 返回测试数据
#     return(test_outfile_title,test_outfile_content)

In [407]:
# 把分词以后的结果写入文件
def writeFile(outputfile,newline):
    
    fw = open(outputfile, 'ab')
    fw.write(newline.encode("utf-8"))
    fw.close()

In [408]:
def run(input_file,ptype):

    # 每次获取最大件数
    STOP_SIZE = 500001
    # 初始化ctr值
    ctr = 0
    
    for i in readDataFile(input_file,ptype):
        if ctr< STOP_SIZE:
            ctr +=1
            print "<%s>" % (ptype) + "第" + str(ctr)+ "件"
            print i[0]

            # if ptype == 'train':label = i[2]
            # if ptype == 'test':doc_id = i[2]
            new_title = "__lable__"+i[2]+ "\t"+ i[0]+ "\n"
            print new_title
            new_content = "__lable__"+i[2]+ "\t" + i[1]+ "\n"
            print new_content
            
            #把出力内容，每10W件数据分别存放
            num = ""
            if ctr <= 100000:
                num = "01"
            if ctr > 100000 and ctr <=200000:
                num = "02"
            if ctr > 200000 and ctr <=300000:
                num = "03"
            if ctr > 300000 and ctr <=400000:
                num = "04"
            if ctr > 400000:
                num = "05"
            
            # 训练数据
            train_outfile_title,train_outfile_content = makeFile(num)
            
            # 测试数据
#             test_outfile_title,test_outfile_content = makeFile(num)
            
            #写入新的文件（训练）
            writeFile(train_outfile_title,new_title)
            writeFile(train_outfile_content,new_content)
            
            #写入新的文件（测试）
#             writeFile(test_outfile_title,new_title)
#             writeFile(test_outfile_content,new_content)

In [409]:
# Main函数
def main():
    input_test = "/home/hadoop/Study/bdci_2017/BDCI2017-360/data/test.tsv"
    # 处理测试文件
    run(input_test,'train')

In [410]:
if __name__ == '__main__':
    main()